# Homework 4 - Hard coding

In [5]:
!pip install ipynb

In [8]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=400d82a1d217f9336fabe3748eae08741d9fb2cb8ee39380563442d94e900679
  Stored in directory: c:\users\parsa33033\appdata\local\pip\cache\wheels\06\7e\ad\1cb03e935234186825cefc7e2c8f3451b4f654b5bc72232a7b
Successfully built import-ipynb


In [ ]:
!pip install librosa

In [46]:
import numpy as np
import pandas as pd
import librosa
import import_ipynb
from AudioSignals import *
import sys,os
import csv
import re
from tqdm import tqdm
import scipy as sp
import time
import pickle as pkl
import hashlib

## 1. Implementing your own Shazam

### 1.1 Getting your data!

We have not used the all the utility functions provided in AudioSignals.ipynb as we thought it was more intuitive to collect wav format of all the songs in one folder called "data" and try to create a dataframe called "dataset" which contains all the information about the songs such as its title, band name, album name, and the path to the track which is in "data" folder.

By this way we could sumulate a database using a csv file named "dataset.csv" and fetch the songs along with their information from this supposed database.

In [532]:
data_dir = "data"
N_TRACKS = 1413

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

def convert_audio(audio_name, audio_path, converted_extention='wav', destination_path="data"):
    destination_audio = os.path.join(destination_path, audio_name[:-3] + converted_extention) 
    if not Path(destination_audio).exists():
        subprocess.check_output(f"ffmpeg -i {audio_path} {destination_audio}", shell=True)
        return destination_audio
    return None

with open("dataset.csv", "w", newline='') as dataset:
    writer = csv.writer(dataset)
    writer.writerow(["title", "band", "album", "track"])
    for path, subdirs, files in tqdm(os.walk("mp3s-32k")):
        for name in files:
            f = path.split('\\')           
            if len(f) == 3:
                mp3_path = os.path.join(path, name)
                wav_path = convert_audio(name, mp3_path)
                if wav_path:
                    writer.writerow([name, f[1], f[2], wav_path])
ds = pd.read_csv("dataset.csv")

100%|█████████████████████████████████████████████████████████████████████████| 1413/1413 [00:00<00:00, 1414113.95it/s]


As we can see from the code above we have done the following:
1. open a csv file called dataset
2. loop over each song in the mp3s-32k, which is the directory containing all the mp3 songs
3. convert the mp3 songs to wav using convert_audio function

### 1.2 Fingerprint hashing

Now we need to create a minhash signature of each song and assign a band of it to an appropriate bucket so that we can find our query tracks using these bands from minhash signatures provided in the buckets.

In [517]:
N_TRACKS = 1413
HOP_SIZE = 512
DURATION = 30 # TODO: to be tuned!
THRESHOLD = 0 # TODO: to be tuned!
data_folder = Path("data/")
tracks = data_folder.glob("*.wav")

In [518]:
hash_sig_num = 100

Now we need to create our hash functions.

As mentinoed in the paper by Kirsch, Adam, and Michael Mitzenmacher with the title of "Less hashing, same performance: Building a better Bloom filter.", if we can create two uniform hash functions, then we can use these to hash functions to create as much other hash functions as we need with the formula below:

$g_{i}(x) = h_{1}(x) + ih_{2}(x) : 0<i \leq hash\_sig\_num$

The hash_sig_num is actually the number of minhash fucntions which will cause the minhash signature array.

In [519]:
m = 62352
c1 = 20899
c2 = 139
def h1(time, freq):
    return (c1 * time +  c2 * freq) % m
def h2(time, freq):
    return (time +  freq) % m
def h(time, freq):
    g = []
    for i in range(hash_sig_num):
        g.append(h1(time, freq) + i * h2(time, freq))
    g = np.array(g, dtype=np.int64)
    return g

As it is evident from the code above we created two hash functions and used them in the formula mentioned. The hash functions are as below:

$h_{1} = (c_{1}time + c_{2}freq) \ Mod \ m $

$h_{2} = (time + freq) \ Mod \ m$

where  $m,c_{1},c_{2}$ are constants

In [520]:
def find_peaks(amplitude):
    freq, time = amplitude.shape
    peaks = []
    for t in range(time):
        s = sp.signal.find_peaks(amplitude[:,t])[0]
        x = np.sum([i * s[i] for i in range(len(s))])
        if x == 0: continue
        peaks.append([t, x])
    peaks = np.array(peaks)
    return peaks

The find_peaks function receives the fourier amplitudes from the spectogram and finds the peaks within each time slot and appends them to a list as a list of time and frequency [t,f].

$f = 1 \times f_{1} + 2 \times f_{2} + 3 \times f_{3} + ... + n \times f_{n}$ 

$result = [[t_{1}, f_{t_{1}}], [t_{2}, f_{t_{2}}], ..., [t_{k}, f_{t_{k}}]]$

The minhash_signauter function hashes each of the elements from the list received from the find_peaks function and creates a minhash signature for each song

In [522]:
def minhash_signature(amplitude):
    peaks = find_peaks(amplitude)
    minhash_sig = []
    minhash_sig = np.min(h(peaks[:,0], peaks[:,1]), axis=1)
    return minhash_sig

#### Now we need to take the dataset.csv and create the minhash signature for each of the songs and create a csv file indicating the minhash signatures related to each song

In [523]:
buckets = dict()
buckets_num = 200
band_num = 10
band_width = hash_sig_num // band_num
band = 4 
band_start = 4 * band_width
band_end = 4 * band_width + band_width

In [524]:
dataset_file_name = 'dataset.csv'
bucket_file_name = 'song_buckets.pkl'
minhash_sig_file_name = 'song_minhash_signature.csv'

In [531]:
t1 = time.time()
with open(dataset_file_name, 'rb') as dataset:
    ds = pd.read_csv(dataset)
    with open(minhash_sig_file_name, 'w', newline="") as minhash_dataset:
        minhash_writer = csv.writer(minhash_dataset)
        minhash_writer.writerow(['index', 'title', 'artist', 'minhash_signature'])
        with open(bucket_file_name, 'wb') as buckets_file:
            for idx, row in tqdm(ds.iterrows(), total=ds.shape[0]):
#                 if idx == 100:
#                     break
                audio = row.track
                artist = row.band
                title = row.title[:-4].split('-')[1]
                track, sr, onset_env, peaks = load_audio_picks(audio, DURATION, HOP_SIZE)
                fft = librosa.stft(track)
                fft_amplitude = librosa.amplitude_to_db(np.abs(fft), ref=np.max)
                minhash_sig = minhash_signature(fft_amplitude)
                minhash_writer.writerow([idx, title, artist, minhash_sig])
                minhash_band = minhash_sig[band_start: band_end]
                bucket = (3 * hash(np.sum(minhash_band)) + 7) % buckets_num
                if bucket in buckets.keys():
                    buckets[bucket].append([idx, minhash_band])
                else:
                    buckets[bucket] = []
                    buckets[bucket].append([idx, minhash_band])
                    
            pkl.dump(buckets, buckets_file)
print("Created minhash signature of " + str(N_TRACKS) + " songs in", "{:.2f}".format(time.time()-t1), "seconds.")

100%|██████████████████████████████████████████████████████████████████████████████| 1412/1412 [25:12<00:00,  1.07s/it]


Created minhash signature of 1413 songs in 1512.55 seconds.


As it is evident from the code above, we are doing the following:

1. read the dataset.csv file in order to fetch the songs in a datafram named "ds"
2. open a song_minhash_signature.csv file in order to write the minhash signature of each song in it.
3. open a song_buckets.pkl file for the dictionary affiliated with the buckets of the bands within each minhash signature
4. loop through each of the items of the dataset "ds"
5. for each song fetch the song and create a minhash signature for it
6. take a band from the signature and assign it as a value to the bucket with the key of the hash of the band's sum.
7. write the minhash signature along with title of the song and name of the artist in the  dataset.csv file in order to fetch the songs in a datafram named "ds" song_minhash_signature.csv

In [ ]:
with open(bucket_file_name, 'rb') as d:
    buckets = pkl.load(d)

In [533]:
for query_num in range(1,11):
    track, sr, onset_env, peaks = load_audio_picks("query/track"+str(query_num)+".wav", DURATION, HOP_SIZE)
    fft = librosa.stft(track)
    fft_amplitude = librosa.amplitude_to_db(np.abs(fft), ref=np.max)
    minhash_sig = minhash_signature(fft_amplitude)
    minhash_band = minhash_sig[band_start: band_end]
    bucket = (3 * hash(np.sum(minhash_band)) + 7) % buckets_num
    t1 = time.time()
    with open(minhash_sig_file_name, 'r', newline="") as minhash_dataset:
        mhds = pd.read_csv(minhash_dataset)
        b = np.array(buckets[bucket])
        for idx, arr in zip(b[:,0], b[:,1]):
            if str(minhash_band) == str(arr):
                if str(mhds['minhash_signature'][idx]) == str(minhash_sig):
                    print("track"+str(query_num)+".wav is ---->",mhds['title'][idx], "by", mhds['artist'][idx],", which was found in", "{:.2f}".format(time.time()-t1), "seconds.")
                    break 


<ipython-input-533-a141aaa96d22>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  b = np.array(buckets[bucket])


track1.wav is ----> Dream_On by aerosmith , which was found in 0.02 seconds.
track2.wav is ----> I_Want_To_Break_Free by queen , which was found in 0.02 seconds.
track3.wav is ----> October by u2 , which was found in 0.02 seconds.
track4.wav is ----> Ob by beatles , which was found in 0.02 seconds.
track5.wav is ----> Karma_Police by radiohead , which was found in 0.02 seconds.
track6.wav is ----> Heartbreaker by led_zeppelin , which was found in 0.02 seconds.
track7.wav is ----> Go_Your_Own_Way by fleetwood_mac , which was found in 0.02 seconds.
track8.wav is ----> American_Idiot by green_day , which was found in 0.02 seconds.
track9.wav is ----> Somebody by depeche_mode , which was found in 0.02 seconds.
track10.wav is ----> Black_Friday by steely_dan , which was found in 0.02 seconds.


1. For the query, we fetch the minhash_signature of the unknown track and take the specific band out. hash the sum of the picked band from the signature and take the hash of the sum.
2. Search the bucket's keys for this hashed value and fetch all the minhash signature bands within that key (each key contains a list of minhash signature bands). 
3. check the picked bucket for all the minhash signatures and find the correct match and give back the title of the song and the name of its artist


As it is shown the tracks' info are as below:


| track #  | artist | song title |
| --- | --- | --- |
| track1.wav | aerosmith | dream on |
| track2.wav | queen  | I Want To Break Free  |
| track3.wav | u2  | October  |
| track4.wav | beatles  | Ob  |
| track5.wav | radiohead  | Karma Police  |
| track6.wav | led zeppelin  | Heartbreaker  |
| track7.wav | fleetwood mac  | Go Your Own Way  |
| track8.wav | green day  | American Idiot  |
| track9.wav | depeche mode  | Somebody  |
| track10.wav | steely dan  | Black Friday  |

## 2. Grouping songs together!

### 2.1 Getting your data!

### 2.2 Choose your features (variables)!

### 2.3 Clustering!

### 2.4 Analysing your results!

## Bonus

## 3. Algorithmic questions

In [720]:
def get_sum_pairs(arr, n, s):
    s_set, l = set(), set()
    for i in range(n):
        s_set.add(arr[i])
        if s - arr[i] in s_set:
            l.add((arr[i], s - arr[i]))
    return l    

In [721]:
A = [7, -2, 8, 2, 6, 4, -7, 2, 1, 3, -3]
get_sum_pairs(A, len(A), 4)

{(-3, 7), (2, 2), (3, 1), (6, -2)}

Since there is one for loop which goes over all the elements of the array only once, the time complexity is of $O(n)$